# A to Z Model

In [ ]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# Paths
dataset = 'model/AtoZ.csv'
model_save_path = 'model/AtoZ.keras'
tflite_save_path = 'model/keypoint_classifier.tflite'

In [ ]:

# Number of classes (A-Z = 26)
NUM_CLASSES = 26

# Load dataset
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2 * 2) + 1)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X_dataset, 
    y_dataset,
    train_size=0.75,
    random_state=RANDOM_SEED
)


In [ ]:
# Create model
model = tf.keras.Sequential([
    tf.keras.layers.Input((84,)),  # 42 features * 2 hands
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal(seed=RANDOM_SEED)),
    tf.keras.layers.Dropout(0.3, seed=RANDOM_SEED),
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal(seed=RANDOM_SEED)),
    tf.keras.layers.Dropout(0.3, seed=RANDOM_SEED),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', kernel_initializer=tf.keras.initializers.HeNormal(seed=RANDOM_SEED))
])

# Compile model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Calculate class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

# Callbacks
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, 
    verbose=1, 
    save_weights_only=False
)
es_callback = tf.keras.callbacks.EarlyStopping(
    patience=20, 
    verbose=1, 
    restore_best_weights=True
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.0001
)


In [ ]:

# Train model
history = model.fit(
    X_train,
    y_train,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    class_weight=class_weight_dict,
    callbacks=[cp_callback, es_callback, reduce_lr]
)

In [ ]:

# Evaluate model
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=32)
print(f"\nValidation accuracy: {val_acc:.4f}")

In [ ]:

# Generate predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)


In [ ]:

# Plot confusion matrix
plt.figure(figsize=(15, 15))
cm = confusion_matrix(y_test, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=[chr(i + 65) for i in range(NUM_CLASSES)],
            yticklabels=[chr(i + 65) for i in range(NUM_CLASSES)])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, 
                          target_names=[chr(i + 65) for i in range(NUM_CLASSES)]))

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Save model as TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()
with open(tflite_save_path, 'wb') as f:
    f.write(tflite_quantized_model)

print("\nTraining and evaluation complete. Model saved as TFLite.")